# Import Module & Define Function

In [2]:
# import the necessary packages
import imutils
import cv2
from matplotlib import pyplot as plt
import numpy
import os
import pandas
import csv
import pickle

## File Operations

In [4]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images

In [5]:
def count_files(path):
    count = 0
    for root_dir, cur_dir, files in os.walk(path):
        count += len(files)
    return count

## Get dictionary key

In [6]:
# function to return key for any value
def get_key_dict(val):
    val_bw = cv2.cvtColor(val, cv2.COLOR_BGR2GRAY)
    val_bin = numpy.interp(val_bw, (val_bw.min(), val_bw.max()), (0, +1))
    for key, dict_list in number_dictionary.items():
        for dict_val in dict_list:
            if numpy.array_equal(val_bin,dict_val):
                return True, key
    return False, 0

In [7]:
def get_current_time(current_frame, last_time, frame):
    d100 = frame[24:31,208:215]
    d10 = frame[24:31,216:223]
    d1 = frame[24:31,224:231]
    
    num1_valid, time_num1 = get_key_dict(d1)
    if not num1_valid:
        return last_time
    num10_valid, time_num10 = get_key_dict(d10)
    if not num10_valid:
        return last_time
    num100_valid, time_num100 = get_key_dict(d100)
    if not num100_valid:
        return last_time

    cur_time = int(f"{time_num100}{time_num10}{time_num1}")
    return cur_time

## Check new info

In [8]:
def new_stage_check(current_frame, last_stage, frame):
    frame_stage_1 = frame[24:31,152:159]
    frame_stage_2 = frame[24:31,168:175]
    num1_valid, stage_num1 = get_key_dict(frame_stage_1)
    num2_valid, stage_num2 = get_key_dict(frame_stage_2)
    if not num1_valid&num2_valid:
        return False, last_stage
    if f"{stage_num1}-{stage_num2}" == last_stage:
        return False, last_stage
    else: 
        return True, f"{stage_num1}-{stage_num2}"

In [9]:
def new_score_check(current_frame, last_score, frame):
    d1 = frame[24:31,48:55]
    d10 = frame[24:31,40:47]
    d100 = frame[24:31,32:39]
    
    num1_valid, score_num1 = get_key_dict(d1)
    if not num1_valid:
        return False, last_score
    num10_valid, score_num10 = get_key_dict(d10)
    if not num10_valid:
        return False, last_score
    num100_valid, score_num100 = get_key_dict(d100)
    if not num100_valid:
        return False, last_score

    ## convert new_score and compare with previous score
    new_score = int(f"{score_num100}{score_num10}{score_num1}")
    if new_score == last_score:
        return False, last_score
    else: 
        return True, new_score
    

In [10]:
def status_down_check(frame_num, counting_time, function_counter, stage, frame):
    cur_time = get_current_time(frame_num, counting_time, frame)
    if (cur_time==stage_limit.get(stage)):
        function_counter=0
        return True, cur_time, function_counter, True
    if (cur_time==(stage_limit.get(stage)-1)):
        return False, cur_time, function_counter, False
    if (function_counter>TIME_THRESHOLD and cur_time==counting_time):
        function_counter=0
        return True, cur_time, function_counter, False
    elif (cur_time != counting_time):
        function_counter=0
        return False, cur_time, function_counter, False
    else:
        function_counter = function_counter+1
        return False, cur_time, function_counter, False

# Loading Dictionary

In [11]:
file_score_display = open("./data_dump/number_dictionary.npy", "rb")
number_dictionary = pickle.load(file_score_display)
print(number_dictionary[1][0])
file_score_display.close()

[[0. 0. 0. 1. 1. 0. 0.]
 [0. 0. 1. 1. 1. 0. 0.]
 [0. 0. 0. 1. 1. 0. 0.]
 [0. 0. 0. 1. 1. 0. 0.]
 [0. 0. 0. 1. 1. 0. 0.]
 [0. 0. 0. 1. 1. 0. 0.]
 [0. 1. 1. 1. 1. 1. 1.]]


In [12]:
stage_limit = {
    '1-1': 400, '1-2': 400, '1-3': 300, '1-4': 300,
    '2-1': 400, '2-2': 400, '2-3': 300, '2-4': 300,
    '3-1': 400, '3-2': 300, '3-3': 300, '3-4': 300,
    '4-1': 400, '4-2': 400, '4-3': 300, '4-4': 400,
    '5-1': 300, '5-2': 400, '5-3': 300, '5-4': 300,
    '6-1': 400, '6-2': 400, '6-3': 300, '6-4': 300,
    '7-1': 400, '7-2': 400, '7-3': 300, '7-4': 400,
    '8-1': 400, '8-2': 400, '8-3': 400, '8-4': 300  ### to check
}

In [13]:
event_dictionary = {
    0: "new_stage",
    1: "flag_reached",
    2: "status_up",
    3: "status_down",
    4: "life_lost"
}

# Comparison & Prediction

In [14]:
participant = 0  ### change for different participant

Version 1

In [138]:
for participant in [0,1,3,5,6,8,9]:
    current_frame = 1
    current_score = 0
    current_stage = "1-1"
    TIME_THRESHOLD=35
    time_frozen = False
    counter = 1
    lock_counter = 0
    counting_time=400
    event_frame = []
    
    print(f"start participant {participant}")
    for frame_number in range(1,count_files(f'game_frame/game_frame_participant_{participant}/')):
        current_time = counting_time            
        # read frame
        frame = cv2.imread(f'game_frame/game_frame_participant_{participant}/game_{frame_number}.png')
        # prediction

        flag_score, new_score = new_score_check(frame_number, current_score, frame)
        flag_stage, next_stage = new_stage_check(frame_number, current_stage, frame)
        flag_time, counting_time, counter, time_frozen_now = status_down_check(frame_number, counting_time, counter, current_stage, frame)

        if flag_stage:
            current_stage = next_stage
            current_score = 0
            lock_counter = 50
            event_frame.append((frame_number-1, next_stage, 0, 1))
            event_frame.append((frame_number, next_stage, 0, 0))
            #print("###### new_stage")
            # print((frame_number, next_stage, 0,0))

        elif flag_score:
            if new_score == 0 and time_frozen_now:  ############# need to add time condition
                event_frame.append((frame_number-1, current_stage, new_score,4))
                #print("####### life_lost")
                # print((frame_number-1, current_stage, new_score,4))
            elif ((new_score-current_score) == 10):
                lock_counter = 0
                event_frame.append((frame_number-6, current_stage, new_score,2))
                #print("###### status up")
                #print((frame_number-6, current_stage, new_score,2))
            current_score = new_score

        elif flag_time:
            if (counting_time == stage_limit.get(current_stage)) and not time_frozen_before and (lock_counter>100):
                event_frame.append((frame_number-1, current_stage, new_score,4))
                # print("####### life_lost")
                # print((frame_number-1, current_stage, new_score,4))

            elif (counting_time == stage_limit.get(current_stage)) and (not(counting_time == stage_limit.get(current_stage))) and not time_frozen_before:
                    event_frame.append((frame_number-1, current_stage, new_score,4))
                    #print("####### life_lost")
                    # print((frame_number-1, current_stage, new_score,4))

            elif not (counting_time == stage_limit.get(current_stage)) and (lock_counter>100):
                lock_counter = 35
                event_frame.append((frame_number-35, current_stage, current_score,3))
                #print("###### status down")
                # print((frame_number-35, current_stage, current_score,3))
                #time_frozen_before = False

        lock_counter += 1
        time_frozen_before = time_frozen_now
        
    print(f"recording participant {participant}")
    
    for frame in event_frame:
        event_list.append([frame[0],event_dictionary[frame[3]]])
        print(f'{frame[0]},{event_dictionary[frame[3]]}')

    with open(f'./game_frame/predictions/game_frame_participant_{participant}.csv', 'w') as f:
        # using csv.writer method from CSV package
        write = csv.writer(f)
        write.writerows(event_list)
        
    print(f"end participant {participant}")
    print(event_frame)

start participant 0
recording participant 0
0,life_lost
428,status_up
1915,flag_reached
1916,new_stage
2003,life_lost
2369,status_up
2545,status_down
3030,status_up
4400,flag_reached
4401,new_stage
4401,life_lost
4694,life_lost
5502,status_up
6055,status_down
6453,flag_reached
6454,new_stage
6454,life_lost
7735,life_lost
9978,flag_reached
9979,new_stage
9979,life_lost
11502,flag_reached
11503,new_stage
11503,life_lost
12626,life_lost
14184,life_lost
15475,life_lost
15695,life_lost
17154,life_lost
19713,flag_reached
19714,new_stage
19997,life_lost
20695,life_lost
20885,life_lost
22313,flag_reached
22314,new_stage
22455,life_lost
22695,life_lost
23641,life_lost
24141,life_lost
24613,life_lost
25699,life_lost
28088,flag_reached
28089,new_stage
28089,life_lost
29214,life_lost
30645,flag_reached
30646,new_stage
31190,life_lost
31714,life_lost
32196,life_lost
32728,life_lost
33292,life_lost
34621,flag_reached
34622,new_stage
35125,life_lost
36686,flag_reached
36687,new_stage
37364,status_up


Version 2

In [ ]:
for participant in [0,1,3,5,6,8,9]:
    current_frame = 1
    current_score = 0
    current_stage = "1-1"
    TIME_THRESHOLD=35
    time_frozen = False
    counter = 1
    lock_counter = 0
    counting_time=400
    event_frame = []
    event_list = []

    print(f"start participant {participant}")
    for frame_number in range(1,count_files(f'game_frame/game_frame_participant_{participant}/')):
        current_time = counting_time            
        # read frame
        frame = cv2.imread(f'game_frame/game_frame_participant_{participant}/game_{frame_number}.png')
        # prediction

        flag_score, new_score = new_score_check(frame_number, current_score, frame)
        flag_stage, next_stage = new_stage_check(frame_number, current_stage, frame)
        flag_time, counting_time, counter, time_frozen_now = status_down_check(frame_number, counting_time, counter, current_stage, frame)

        if flag_stage:
            current_stage = next_stage
            current_score = 0
            lock_counter = 50
            event_frame.append((frame_number-1, next_stage, 0, 1))
            event_frame.append((frame_number, next_stage, 0, 0))
            #print("###### new_stage")
            print((frame_number, next_stage, 0,0))

        elif flag_score:
            if new_score == 0 and time_frozen_now:  ############# need to add time condition
                event_frame.append((frame_number-1, current_stage, new_score,4))
                #print("####### life_lost")
                print((frame_number-1, current_stage, new_score,4))
            elif ((new_score-current_score) == 10):
                lock_counter = 0
                event_frame.append((frame_number-6, current_stage, new_score,2))
                #print("###### status up")
                print((frame_number-6, current_stage, new_score,2))
            current_score = new_score

        elif flag_time:
            if (lock_counter>100):
                if (counting_time == stage_limit.get(current_stage)):
                    if not time_frozen_before:
                        # print("####### life_lost")
                        print((frame_number-1, current_stage, new_score,4))
                        event_frame.append((frame_number-1, current_stage, new_score,4))
                else:
                    lock_counter = 35
                    event_frame.append((frame_number-35, current_stage, current_score,3))
                    #print("###### status down")
                    print((frame_number-35, current_stage, current_score,3))

        lock_counter += 1
        time_frozen_before = time_frozen_now

    print(f"recording participant {participant}")

    for frame in event_frame:
        event_list.append([frame[0],event_dictionary[frame[3]]])
        print(f'{frame[0]},{event_dictionary[frame[3]]}')

    with open(f'./game_frame/predictions/game_frame_participant_{participant}.csv', 'w') as f:
        # using csv.writer method from CSV package
        write = csv.writer(f)
        write.writerows(event_list)

    print(f"end participant {participant}")
    print(event_frame)

start participant 0
(428, '1-1', 12, 2)
(1916, '1-2', 0, 0)
(2003, '1-2', 0, 4)
(2369, '1-2', 10, 2)
(2545, '1-2', 10, 3)
(3030, '1-2', 41, 2)
(4401, '1-3', 0, 0)
(4694, '1-3', 0, 4)
(5502, '1-3', 17, 2)
(6055, '1-3', 26, 3)
(6454, '2-2', 0, 0)
(7735, '2-2', 0, 4)
(9979, '1-4', 0, 0)
(11503, '3-1', 0, 0)
(12626, '3-1', 0, 4)
(14184, '3-1', 0, 4)
(15475, '3-1', 0, 4)
(15695, '3-1', 0, 4)
(17154, '3-1', 0, 4)
(19714, '4-1', 0, 0)
(19997, '4-1', 0, 4)
(20695, '4-1', 0, 4)
(20885, '4-1', 0, 4)
(22314, '2-1', 0, 0)
(22455, '2-1', 0, 4)
(22695, '2-1', 0, 4)
(23641, '2-1', 0, 4)
(24141, '2-1', 0, 4)
(24613, '2-1', 0, 4)
(25699, '2-1', 0, 4)
(28089, '2-3', 0, 0)
(29214, '2-3', 0, 4)
(30646, '2-4', 0, 0)
(31190, '2-4', 0, 4)
(31714, '2-4', 0, 4)
(32196, '2-4', 0, 4)
(32728, '2-4', 0, 4)
(33292, '2-4', 0, 4)
